## The Knapsack Problem

This notebook contains different algorithms to solve the knapsack problem. The knapsack problem is defined as follows:

- let there be $n$ items denoted by $i = 0,..., n-1$
- let $v_i$ be value of an item and the $w_i$ weight
- let $K$ be the capacity of the knapsack
- let $x_i$ be a variable that takes the valie $1$ if item $i$ is taken in the knapsack and $0$ otherwise

The goal is to find the $k$ items to take in the knapsack that maximise the total value. The problem is formalised as: 
<br>
<br>
<br>
$$\begin{alignat}{3}
\text{maximise: }                &\quad&  \sum_{i \in 0, ..., n-1} v_i \cdot x_i  &&          & \\
\text{subject to: } &\quad&  \sum_{i \in 0, ..., n-1} w_i \cdot x_i  && \leq K   &\\
                    &\quad&  x_i \in {\{0,1\}}                       &&    &\quad \forall i=0,...,n-1
\end{alignat}\\
$$
<br>
<br>

The algorithms implemented to solve the problem are:
- greedy algorithm 
- dynamic program
- branch and bound

In [1]:
from knapsack.problem_setup import parse_input_data, output_solution
from knapsack.dynamic_program import dp_solver
from knapsack.greedy import greedy_solver
from knapsack.branch_and_bound import BranchAndBound

Different knapsack files from the *data* folder can be entered in the cell below to test the algorithms on different problems. 

In [2]:
with open('data\ks_50_0', 'r') as input_data_file:
    input_data = input_data_file.read()
    
items, capacity = parse_input_data(input_data)

print("The knapsack has a capacity of {:,.0f} and there are {:,.0f} items to be considerd".format(capacity, len(items)))

The knapsack has a capacity of 341,045 and there are 50 items to be considerd


### Greedy Algorithm

In most cases, greedy algorithms are unable achieve optimality. However, they are useful for quickly finding a solution that can be used as a baseline with which to compare solutions from more complex algorithms; here is no exception.

The greediness of this algorithm is in relation to the value to weight ration of each item. That is, starting with the item that has the most value per weight units, items are taken progressively until the knapsack's capacity is filled.

Due to its simplicity, the algorithm is able to very quickly arrive at a feasible solution, even for very large problems, but without any guarantee of optimality.

In [3]:
decision_variables_gr, obj_value_gr = greedy_solver(items, capacity)
selected_items_gr = [idx for idx, var in enumerate(decision_variables_gr) if var == 1]

print("The knapsack has a total value of {:,.0f} and contains:".format(obj_value_gr))
for item in selected_items_gr:
    print("- item {:,.0f} with weight {:,.0f} and value {:,.0f}".format(item, items[item].weight, items[item].value))

The knapsack has a total value of 141,956 and contains:
- item 0 with weight 4,912 and value 1,906
- item 6 with weight 1,484 and value 492
- item 19 with weight 71,276 and value 29,688
- item 22 with weight 231,302 and value 96,601
- item 33 with weight 32,038 and value 13,269


### Dynamic Program

The dynamic program implementation for the knapsack problem is guaranteed to find an exact solution. However, with its "pseudo-polynomial" time complexity of $O(nK)$ it can perform poorly for problems with large number of potential items and/or a large capacity.  

This can be seen with the problems in the *data* folder. The dynamic program quickly solves small to medium sized problems such as the instance with 1,000 potential items and a capacity of 100,000.

In [4]:
decision_variables_dp, obj_value_dp = dp_solver(items, capacity)
selected_items_dp = [idx for idx, var in enumerate(decision_variables_dp) if var == 1]

print("The knapsack has a total value of {:,.0f} and contains:".format(obj_value_dp))
for item in selected_items_dp:
    print("- item {:,.0f} with weight {:,.0f} and value {:,.0f}".format(item, items[item].weight, items[item].value))

The knapsack has a total value of 142,156 and contains:
- item 4 with weight 108,372 and value 45,136
- item 22 with weight 231,302 and value 96,601
- item 44 with weight 1,338 and value 419


But for larger problems, such as the one with 400 items and a capacity of 9,486,367 or the one with 10,000 items and a capacity of 1,000,000, the time and space complexity reach a level where an optimal solution cannot be found within a practical period of time. Thus, a computationally more efficient alternative is needed for such problems.

### Branch and Bound

Unlike dynamic programming, the branch and bound does not perform an exhaustive search. Instead, the algorithm creates a decision tree like structure to more intelligently search through the possible combinations. 

To begin with, a "relaxed" solution is calculated, in which a constraint is relaxed to give an upper bound on the possible solution. Here, a linear relaxation was applied. The integrality constraint for the items is relaxed, meaning that a portion of an item can be taken. This relaxation would allow a configuration better than any possible solution with the integrality constraint. 

At each level of the tree, a solution is calculated, in which the the variables up to the current level are set according to the preceding branches and the remaining variables are assigned under the relaxed conditions. The relaxed configuration is checked for feasibility (according to the knapsack weight) and to see if it beats the current best (valid) solution. If it fails either of these checks, the tree is bounded at that point and the algorithm moves on to the next branch. 

The process is repeated until the entire decision space has either been searched or bounded off and an optimal solution is found; a best first search approach was used here. Branch and bound performs considerably better than dynamic programming on large programs by skipping sections of the decision space that are infeasible or will not improve the current best result. 

The 400 item problem that the dynamic program is unable to solve can be solved fairly quickly using branch and bound.

In [5]:
with open('data\ks_400_0', 'r') as input_data_file:
    input_data = input_data_file.read()
    
items, capacity = parse_input_data(input_data)

print("The knapsack has a capacity of {:,.0f} and there are {:,.0f} items to be considerd".format(capacity, len(items)))

The knapsack has a capacity of 9,486,367 and there are 400 items to be considerd


In [6]:
bnb = BranchAndBound(input_data)
decision_variables_bnb, obj_value_bnb = bnb.solve()
selected_items_bnb = [idx for idx, var in enumerate(decision_variables_bnb) if var == 1]

print("The knapsack has a total value of {:,.0f} and contains:".format(obj_value_dp))
for item in selected_items_bnb:
    print("- item {:,.0f} with weight {:,.0f} and value {:,.0f}".format(item, items[item].weight, items[item].value))

The knapsack has a total value of 142,156 and contains:
- item 95 with weight 136,076 and value 56,788
- item 313 with weight 273,812 and value 114,356
- item 374 with weight 3,265,540 and value 1,365,820
- item 376 with weight 3,457,100 and value 1,445,900
- item 379 with weight 1,177,692 and value 492,496
- item 393 with weight 1,176,140 and value 491,820
